In [1]:
import json
import requests
import websockets
import asyncio

# Apply public connect token

In [2]:
url = 'https://api.kucoin.com/api/v1/bullet-public'

In [3]:
x = requests.post(url)

In [4]:
cjson = json.loads(x.text)

In [5]:
endpoint = cjson['data']['instanceServers'][0]['endpoint']
token = cjson['data']['token']

# Subscription

In [6]:
sub = {
    "id": 1545910660739,
    "type": "subscribe",
    "topic": "/market/ticker:XMR-BTC",
    "response": True
}

# Single Websocket Connection

In [7]:
async def connection():
    async with websockets.connect(endpoint+"?token="+token) as conn: # creates ws connection
        await conn.send(json.dumps(sub)) # sends ws message
        res = await conn.recv() # recieves ws message
        counter = 0
        while True and counter < 2:
            res = await conn.recv()
            print(res)
            counter += 1
        await conn.close()

In [8]:
multi = {"id": "1Jpg30DEdU", "type": "openTunnel", "newTunnelId": "bt1", "response": False}
# command used to open tunnel

In [9]:
multi_spotEth = {"id": "1JpoPamgFM",
             "type": "subscribe",
             "topic": "/market/ticker:ETH-USDT",
             "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

In [10]:
multi_futureEth = {"id": 1545910660740,                          
                "type": "subscribe",
                "topic": "/contractMarket/tickerV2:ETHUSDTM",
                "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

<h3>Multiplex Websocket Connection</h3>

In [11]:
async with websockets.connect(endpoint+"?token="+token) as conn:
    await conn.send(json.dumps(multi))  # initial open tunel command
    res = await conn.recv()
    
    await conn.send(json.dumps(multi_spotEth)) # send tunnel subscription
    res = await conn.recv()
    await conn.send(json.dumps(multi_futureEth))  # send tunnel subscription
    res = await conn.recv()
    
    counter = 0
    spotPrice = 0
    futurePrice = 0
    while True and counter < 5:
        res = await conn.recv()
        vals = json.loads(res)

        if (vals['topic']) == '/contractMarket/tickerV2:ETHUSDTM':
#             print(f"Future Price is: {vals['data']['bestAskPrice']}")
            futurePrice = float(vals['data']['bestAskPrice'])

        elif (vals['topic']) == '/market/ticker:ETH-USDT':
#             print(f"Spot Price is: {vals['data']['bestAsk']}")
            spotPrice = float(vals['data']['bestAsk'])
    
        print(spotPrice - futurePrice)
        
        if spotPrice - futurePrice > 0.8 and futurePrice != 0 and spotPrice != 0:
            print('Long Future and Short Spot')
            
        elif futurePrice - spotPrice > 0.8 and futurePrice != 0 and spotPrice != 0:
            print('Long Spot and Short Future')
            
        counter += 1
    eth_tunnel = {"id": "1JpoPamgFM", "type": "closeTunnel", "tunnelId": "bt1", "response": False}            
    btc_tunnel = {"id": "1JpoPamgFN", "type": "closeTunnel", "tunnelId": "bt1", "response": False}
    await conn.send(json.dumps(eth_tunnel))
    await conn.send(json.dumps(btc_tunnel))
    await conn.close()

-1734.8
0.13000000000010914
0.13000000000010914
0.13000000000010914
0.13000000000010914


# Trying to Long the underpriced and short the overpriced